In [7]:
import re
import pickle
import requests
import pandas as pd

from datetime import datetime, timedelta
from tqdm import tqdm

In [8]:
with open('data/raw.pkl', 'rb') as f:
    df = pickle.load(f)

df.head()

,title,published_at,description,tags,thumbnail,duration,view_count,like_count,comment_count,transcription
id,,,,,,,,,,
8N3sFRR9-OE,How bad is the Cheapest Laptop,2023-07-12T17:23:57Z,"Check out the UGREEN Nexode 100W Charger, 145W...","[Cheapest Laptop, AliExpress]",https://i.ytimg.com/vi/8N3sFRR9-OE/default.jpg,PT23M49S,453842,25856,1891,[{'text': 'I've got something I want to show y...
-n8N62DeNDU,WHY is Everyone Buying This Power Supply??,2023-07-11T17:03:27Z,Checkout iFixit's toolkits at: https://www.iFi...,"[Power Supply, PSU Tester, Thermaltake Smart 6...",https://i.ytimg.com/vi/-n8N62DeNDU/default.jpg,PT14M39S,1116918,51871,2983,[{'text': 'this is the most popular power supp...
buLyy7x2dcQ,"Apple fans, start typing your angry comments now…",2023-07-10T18:14:04Z,"Check out the UGREEN PowerRoam 1200W, 145W Pow...","[apple, mac, mac studio, apple silicon, m2, m2...",https://i.ytimg.com/vi/buLyy7x2dcQ/default.jpg,PT18M25S,1728263,70674,4720,[{'text': 'I really did try this time guys I e...
H5e3ALqgpaA,I said YES to everything… I regret it,2023-07-09T17:20:35Z,Visit https://www.squarespace.com/LTT and use ...,"[saying yes, roundup, tech, assorted, cuktech,...",https://i.ytimg.com/vi/H5e3ALqgpaA/default.jpg,PT26M11S,1823313,76297,3044,[{'text': 'my inbox is full of opportunities t...
P32OKr74NPQ,Upgrading our FREE internet to 25 gigabit!,2023-07-08T17:00:29Z,It’s no secret their chairs are great! Check o...,None,https://i.ytimg.com/vi/P32OKr74NPQ/default.jpg,PT32M19S,1835615,68739,2771,[{'text': 'when we expanded our space to give ...


# Missing values

First we will check for missing values

In [9]:
df.isna().sum(axis=0)

title             0
published_at      0
description       0
tags             81
thumbnail         0
duration          0
view_count        0
like_count        0
comment_count     0
transcription     1
dtype: int64

As we can see we have 9% of the videos without tags and only 1 video without a transcription. We will keep this in mind during the analysis, but no need to go back to to the collection phase.

# Variable types

In [10]:
df.dtypes

title            object
published_at     object
description      object
tags             object
thumbnail        object
duration         object
view_count       object
like_count       object
comment_count    object
transcription    object
dtype: object

We want:

- `published_at` to be a `datetime`
- `duration` to be an `timedelta`

In [11]:
def parse_duration(text):
    '''
    '''
    h_search = re.search('(\d+)H', text)
    m_search = re.search('(\d+)M', text)
    s_search = re.search('(\d+)S', text)

    h = int(h_search.group(1) if h_search else 0)
    m = int(m_search.group(1) if m_search else 0)
    s = int(s_search.group(1) if s_search else 0)

    return timedelta(hours=h, minutes=m, seconds=s)

# visually inspected to match
# df['parsed_duration'] = df.duration.apply(parse_duration)
# with pd.option_context('display.max_rows', None):
#    display(df[['duration', 'parsed_duration']])

df['published_at'] = pd.to_datetime(df.published_at)
df['duration'] = df.duration.apply(parse_duration)

df.head()

## Parsing the transcription

The transcriptions are in a list where every entry is a dictionary with the following fields: `text`, `start`, and `duration`.

Since one of the analysis we want to is the relationship between the script and video performance, we need to extract the `text`. I also have the hypothesis that the beginning of the video has a higher weight (namely before the sponsored segment), so we will extract the script at 1, 2, 5, and 10 minutes. We will also extract the full script.

\* We will also use some Natural Language Processing techniques to process the text later

In [41]:
def parse_transcription(lines):
    '''
        lines: [{}]
        Since this is a one time thing, we will not try to optimize this
    '''
    if not lines:
        return [None] * 5
        
    before_1min = ' '.join([line.get('text') for line in lines if line.get('start') < 60])
    before_2min = ' '.join([line.get('text') for line in lines if line.get('start') < 2 * 60])
    before_5min = ' '.join([line.get('text') for line in lines if line.get('start') < 5 * 60])
    before_10min = ' '.join([line.get('text') for line in lines if line.get('start') < 10 * 60])
    all = ' '.join([line.get('text') for line in lines])

    return before_1min, before_2min, before_5min, before_10min, all
        

transcription_cols = ['1min_text', '2min_text', '5min_text', '10min_text', 'full_text']
df[transcription_cols] = df.apply(lambda row: parse_transcription(row.transcription),
                                  axis=1,
                                  result_type='expand')

df[transcription_cols].head()

,1min_text,2min_text,5min_text,10min_text,full_text
id,,,,,
8N3sFRR9-OE,I've got something I want to show you meet my ...,I've got something I want to show you meet my ...,I've got something I want to show you meet my ...,I've got something I want to show you meet my ...,I've got something I want to show you meet my ...
-n8N62DeNDU,this is the most popular power supply on the m...,this is the most popular power supply on the m...,this is the most popular power supply on the m...,this is the most popular power supply on the m...,this is the most popular power supply on the m...
buLyy7x2dcQ,I really did try this time guys I even went as...,I really did try this time guys I even went as...,I really did try this time guys I even went as...,I really did try this time guys I even went as...,I really did try this time guys I even went as...
H5e3ALqgpaA,my inbox is full of opportunities to review ev...,my inbox is full of opportunities to review ev...,my inbox is full of opportunities to review ev...,my inbox is full of opportunities to review ev...,my inbox is full of opportunities to review ev...
P32OKr74NPQ,when we expanded our space to give the Creator...,when we expanded our space to give the Creator...,when we expanded our space to give the Creator...,when we expanded our space to give the Creator...,when we expanded our space to give the Creator...


## Thumbnails

We also want to download all the thumbnails, as we intend to use them to check for relationships with performance.

In [76]:
for (id, src) in tqdm(zip(df.index, df.thumbnail)):
    response = requests.get(src)
    
    if response.status_code != 200:
        print(f'Could not download {id=} ({src=})')
        continue
    
    with open(f'data/thumbnails/{id}.jpg', 'wb') as f:
        f.write(response.content)

900it [04:19,  3.47it/s]


# Draft

In [52]:
ans = requests.get('https://i.ytimg.com/vi/8N3sFRR9-OE/default.jpg')

In [53]:
with open(f'data/thumbnails/{id}.jpg', 'wb') as f:
    f.write(ans.content)

In [43]:
df.columns

Index(['title', 'published_at', 'description', 'tags', 'thumbnail', 'duration',
       'view_count', 'like_count', 'comment_count', 'transcription',
       'before_1min', 'before_2min', 'before_5min', 'before_10min',
       'full_transcription', 'text_1min', 'text_2min', 'text_5min',
       'text_10min', 'text_full', '1min_text', '2min_text', '5min_text',
       '10min_text', 'full_text'],
      dtype='object')

In [33]:
a = df[:10].copy(deep=True)

a[['b1', 'b2', 'b5', 'b10', 'all']] = a.apply(lambda row: parse_transcription(row.transcription), axis=1, result_type='expand')

In [34]:
a

,title,published_at,description,tags,thumbnail,duration,view_count,like_count,comment_count,transcription,b1,b2,b5,b10,all
id,,,,,,,,,,,,,,,
8N3sFRR9-OE,How bad is the Cheapest Laptop,2023-07-12 17:23:57+00:00,"Check out the UGREEN Nexode 100W Charger, 145W...","[Cheapest Laptop, AliExpress]",https://i.ytimg.com/vi/8N3sFRR9-OE/default.jpg,0 days 00:23:49,453842,25856,1891,[{'text': 'I've got something I want to show y...,I've got something I want to show you meet my ...,I've got something I want to show you meet my ...,I've got something I want to show you meet my ...,I've got something I want to show you meet my ...,I've got something I want to show you meet my ...
-n8N62DeNDU,WHY is Everyone Buying This Power Supply??,2023-07-11 17:03:27+00:00,Checkout iFixit's toolkits at: https://www.iFi...,"[Power Supply, PSU Tester, Thermaltake Smart 6...",https://i.ytimg.com/vi/-n8N62DeNDU/default.jpg,0 days 00:14:39,1116918,51871,2983,[{'text': 'this is the most popular power supp...,this is the most popular power supply on the m...,this is the most popular power supply on the m...,this is the most popular power supply on the m...,this is the most popular power supply on the m...,this is the most popular power supply on the m...
buLyy7x2dcQ,"Apple fans, start typing your angry comments now…",2023-07-10 18:14:04+00:00,"Check out the UGREEN PowerRoam 1200W, 145W Pow...","[apple, mac, mac studio, apple silicon, m2, m2...",https://i.ytimg.com/vi/buLyy7x2dcQ/default.jpg,0 days 00:18:25,1728263,70674,4720,[{'text': 'I really did try this time guys I e...,I really did try this time guys I even went as...,I really did try this time guys I even went as...,I really did try this time guys I even went as...,I really did try this time guys I even went as...,I really did try this time guys I even went as...
H5e3ALqgpaA,I said YES to everything… I regret it,2023-07-09 17:20:35+00:00,Visit https://www.squarespace.com/LTT and use ...,"[saying yes, roundup, tech, assorted, cuktech,...",https://i.ytimg.com/vi/H5e3ALqgpaA/default.jpg,0 days 00:26:11,1823313,76297,3044,[{'text': 'my inbox is full of opportunities t...,my inbox is full of opportunities to review ev...,my inbox is full of opportunities to review ev...,my inbox is full of opportunities to review ev...,my inbox is full of opportunities to review ev...,my inbox is full of opportunities to review ev...
P32OKr74NPQ,Upgrading our FREE internet to 25 gigabit!,2023-07-08 17:00:29+00:00,It’s no secret their chairs are great! Check o...,None,https://i.ytimg.com/vi/P32OKr74NPQ/default.jpg,0 days 00:32:19,1835615,68739,2771,[{'text': 'when we expanded our space to give ...,when we expanded our space to give the Creator...,when we expanded our space to give the Creator...,when we expanded our space to give the Creator...,when we expanded our space to give the Creator...,when we expanded our space to give the Creator...
diUOdC2tYzM,I Want an iPhone because iOS 17 Looks AMAZING,2023-07-06 16:58:35+00:00,Save time and automate your social media marke...,"[iOS 17, Android, Apple, iPhone, iOS, iPad, Ap...",https://i.ytimg.com/vi/diUOdC2tYzM/default.jpg,0 days 00:13:02,1822896,75882,5049,[{'text': 'just kidding it's me Yvonne I need ...,just kidding it's me Yvonne I need help with o...,just kidding it's me Yvonne I need help with o...,just kidding it's me Yvonne I need help with o...,just kidding it's me Yvonne I need help with o...,just kidding it's me Yvonne I need help with o...
QwIlhuR_N2g,The Scariest Fan ever attached to a computer,2023-07-05 17:13:58+00:00,Looking for electronic components and equipmen...,"[Massive Fan, Highest Airflow PC, VF-390, Vasy...",https://i.ytimg.com/vi/QwIlhuR_N2g/default.jpg,0 days 00:19:39,1823271,89575,7295,[{'text': 'this is the vf390 a 390 millimeter ...,this is the vf390 a 390 millimeter fan that is...,this is the vf390 a 390 millimeter fan that is...,this is the vf390 a 390 millimeter fan that is...,this is the vf390 a 390 millimeter fan that is...,this is the vf390 a 3

In [18]:
type(df.iloc[0].transcription[0].get('start'))

float

In [20]:
pd.DataFrame(df.iloc[0].transcription)

,text,start,duration
0,I've got something I want to show you,0.000,5.299
1,meet my big pink 10 inch,1.620,7.920
2,Netbook from AliExpress why you ask if,5.299,7.001
3,you sort all the new Windows laptops in,9.540,5.099
4,existence from low to high the top,12.300,6.180
...,...,...,...
574,maybe check out that dual screen,1419.539,4.201
575,AliExpress laptop that we checked out,1421.640,4.080
576,recently where our community actually,1423.740,3.299
577,figured out a bunch of stuff on the,1425.720,5.000


In [7]:
import re

re.findall('PT(\d+)M(\d+)S', 'PT9M1S')

[('9', '1')]

In [30]:
pattern = re.compile(r'(\d+)M')

a = re.search(r'(\d+)M', 'PT9M')

In [32]:
a.group(1)

'9'

In [19]:
df.published_at

0      2023-07-12T17:23:57Z
1      2023-07-11T17:03:27Z
2      2023-07-10T18:14:04Z
3      2023-07-09T17:20:35Z
4      2023-07-08T17:00:29Z
               ...         
895    2020-08-26T07:00:21Z
896    2020-08-25T18:27:13Z
897    2020-08-24T16:59:58Z
898    2020-08-23T17:11:42Z
899    2020-08-22T17:57:21Z
Name: published_at, Length: 900, dtype: object

In [18]:
pd.to_datetime(df.published_at)

0     2023-07-12 17:23:57+00:00
1     2023-07-11 17:03:27+00:00
2     2023-07-10 18:14:04+00:00
3     2023-07-09 17:20:35+00:00
4     2023-07-08 17:00:29+00:00
                 ...           
895   2020-08-26 07:00:21+00:00
896   2020-08-25 18:27:13+00:00
897   2020-08-24 16:59:58+00:00
898   2020-08-23 17:11:42+00:00
899   2020-08-22 17:57:21+00:00
Name: published_at, Length: 900, dtype: datetime64[ns, UTC]

In [10]:
df[df.drop('tags', axis=1).isna().any(axis=1)]

,id,title,published_at,description,tags,thumbnail,duration,view_count,like_count,comment_count,transcription
708,dJwjqZZgcWk,LTT has a New Home... (April Fools 2021),2021-04-01T13:50:12Z,Get a 15-day free trial for unlimited backup a...,"['april fools', 'onlyfans', 'only fans']",https://i.ytimg.com/vi/dJwjqZZgcWk/default.jpg,PT4M36S,1697153,191214,27458,NaN


In [7]:
df[df.published_at.isna()]

,id,title,published_at,description,tags,thumbnail,duration,view_count,like_count,comment_count,transcription


In [11]:
df.tail()

,id,title,views,time_ago,transcription
25,2fKIaalk4_w,I should stop building computers.,1.6M views,1 month ago,[{'text': 'last year we put together an absolu...
26,QgrVVyIzecM,There’s NO WAY this works - Debunking bogus ne...,2.5M views,1 month ago,[{'text': 'on the surface this is the most sen...
27,bY4MTjVEtjE,There goes all my phone sponsorships…,2.2M views,1 month ago,[{'text': 'after 10 days of construction our b...
28,SqB0lUcqFbA,Hating Apple is Getting REALLY Hard - WWDC 2023,3M views,1 month ago,[{'text': 'Apple just demolished the competiti...
29,O8gCqSmtkrM,This Broke One Week out of Warranty... Can I F...,1.3M views,1 month ago,[{'text': 'this is one sad man he was just one...


In [49]:
import re

# Since we can't use months and years in timedelta
# and we're only approximating the create_date
# this approach converts it to days with the lambda
time_patterns = {
    'hours': (r'(\d)\s+hours?', lambda x: x / 24),
    'days': (r'(\d)\s+days?', lambda x: x),
    'months': (r'(\d)\s+months?', lambda x: x * 30),
    'years': (r'(\d)\s+years?', lambda x: x * 365)
}

def parse_time(text, rel_date=datetime.today()):
    '''
    Given what YouTube shows, returns the approximate date of publication

    It seems that there isn't a mix of patterns, so we will loop and return the
    first match
    '''
    for _, (pattern, f) in time_patterns.items():
        match = re.search(pattern, text)

        if match:
            amount = f(int(match.group(1)))
            
            return rel_date - timedelta(days=amount)
    

In [50]:
df['create_date'] = [parse_time(t) for t in df.time_ago]

In [51]:
df

,id,title,views,time_ago,transcription,timededlta,create_date
0,buLyy7x2dcQ,"Apple fans, start typing your angry comments now…",1.2M views,7 hours ago,[{'text': 'I really did try this time guys I e...,1,2023-07-11 14:55:18.952608
1,H5e3ALqgpaA,I said YES to everything… I regret it,1.6M views,1 day ago,[{'text': 'my inbox is full of opportunities t...,1,2023-07-10 21:55:18.952608
2,P32OKr74NPQ,Upgrading our FREE internet to 25 gigabit!,1.7M views,2 days ago,[{'text': 'when we expanded our space to give ...,1,2023-07-09 21:55:18.952608
3,diUOdC2tYzM,I Want an iPhone because iOS 17 Looks AMAZING,1.7M views,3 days ago,"[{'text': ""just kidding it's me Yvonne I need ...",1,2023-07-08 21:55:18.952608
4,QwIlhuR_N2g,The Scariest Fan ever attached to a computer,1.7M views,5 days ago,[{'text': 'this is the vf390 a 390 millimeter ...,1,2023-07-06 21:55:18.952608
5,JVBnJtzEuI0,You'll regret spamming me...,2M views,6 days ago,[{'text': 'we get loads of emails from everyon...,1,2023-07-05 21:55:18.952608
6,FLG08-xJGyM,This WEIRD Japanese Laptop LOOKS 20 Years Old…,1.6M views,7 days ago,"[{'text': ""this laptop came out in 2022 and it...",1,2023-07-04 21:55:18.952608
7,yI7fV88T8A0,Apple's new Mac Pro can't do THIS!,1.2M views,8 days ago,[{'text': 'I have three apples that do not sup...,1,2023-07-03 21:55:18.952608
8,ZGy2kkWOxig,The Compensation PC,1.4M views,9 days ago,[{'text': 'wow look at the size of this comput...,1,2023-07-02 21:55:18.952608
9,_CxkmtBqGn8,Our Studio was a HUGE waste of money - New Cha...,2.3M views,10 days ago,[{'text': 'you guys are not going to believe w...,1,2023-07-11 21:55:18.952608


In [36]:
parse_time(df.iloc[0].time_ago)

7:00:00


datetime.datetime(2023, 7, 11, 14, 45, 24, 973490)

In [24]:
matches.group(1)

'7'

In [10]:
df.iloc[0].time_ago

'7 hours ago'

In [4]:
from datetime import datetime, timedelta

In [6]:
timedelta(df.iloc[0].time_ago)

TypeError: unsupported type for timedelta days component: str

For the first step we will clear the `views`, transforming them into numbers.

In [4]:
def parse_views(text):
    '''
    There seems to be a couple of cases:
        - Contains dot
            - 8.8K -> Replace '.' with '' -> replace 'K' with '00'
            - 1.4M -> Replace '.' with '' -> replace 'M' with '00000'
        - Without dots
            - 394K -> Replace 'K' with '000'
            - 2M -> Replace 'M' with '000000'
            
    Then we remove the 'views', strip and parse as int
    '''
    if '.' in text:
        k_replace = '00'
        m_replace = '00000'
    else:
        k_replace = '000'
        m_replace = '000000'

    views = int(text\
                .replace('.', '')\
                .replace('K', k_replace)\
                .replace('M', m_replace)\
                .replace('views', '')\
                .strip())
    
    return views

df['clean_views'] = [parse_views(view_text) for view_text in df.views]

In [5]:
df.head()

,id,title,views,transcription,clean_views
0,buLyy7x2dcQ,"Apple fans, start typing your angry comments now…",1.2M views,[{'text': 'I really did try this time guys I e...,1200000
1,H5e3ALqgpaA,I said YES to everything… I regret it,1.6M views,[{'text': 'my inbox is full of opportunities t...,1600000
2,P32OKr74NPQ,Upgrading our FREE internet to 25 gigabit!,1.7M views,[{'text': 'when we expanded our space to give ...,1700000
3,diUOdC2tYzM,I Want an iPhone because iOS 17 Looks AMAZING,1.7M views,"[{'text': ""just kidding it's me Yvonne I need ...",1700000
4,QwIlhuR_N2g,The Scariest Fan ever attached to a computer,1.7M views,[{'text': 'this is the vf390 a 390 millimeter ...,1700000


In [7]:
df

,id,title,views,transcription
0,buLyy7x2dcQ,"Apple fans, start typing your angry comments now…",1.2M views,[{'text': 'I really did try this time guys I e...
1,H5e3ALqgpaA,I said YES to everything… I regret it,1.6M views,[{'text': 'my inbox is full of opportunities t...
2,P32OKr74NPQ,Upgrading our FREE internet to 25 gigabit!,1.7M views,[{'text': 'when we expanded our space to give ...
3,diUOdC2tYzM,I Want an iPhone because iOS 17 Looks AMAZING,1.7M views,"[{'text': ""just kidding it's me Yvonne I need ..."
4,QwIlhuR_N2g,The Scariest Fan ever attached to a computer,1.7M views,[{'text': 'this is the vf390 a 390 millimeter ...
5,JVBnJtzEuI0,You'll regret spamming me...,2M views,[{'text': 'we get loads of emails from everyon...
6,FLG08-xJGyM,This WEIRD Japanese Laptop LOOKS 20 Years Old…,1.6M views,"[{'text': ""this laptop came out in 2022 and it..."
7,yI7fV88T8A0,Apple's new Mac Pro can't do THIS!,1.2M views,[{'text': 'I have three apples that do not sup...
8,ZGy2kkWOxig,The Compensation PC,1.4M views,[{'text': 'wow look at the size of this comput...
9,_CxkmtBqGn8,Our Studio was a HUGE waste of money - New Cha...,2.3M views,[{'text': 'you guys are not going to believe w...
